In [304]:
import pandas as pd

from sklearn import preprocessing
from sklearn import linear_model
from sklearn import ensemble
from sklearn import tree

In [305]:
train = pd.read_csv('train.csv')

In [306]:
features = ['SG_UF_RESIDENCIA',
           'NU_IDADE',
           'TP_SEXO',
           'TP_COR_RACA',
           'TP_NACIONALIDADE',
           'TP_ST_CONCLUSAO',
           'TP_ANO_CONCLUIU',
           'TP_ESCOLA',
           'TP_ENSINO',
           'IN_TREINEIRO',
           'TP_DEPENDENCIA_ADM_ESC',
           'IN_BAIXA_VISAO',
           'IN_CEGUEIRA',
           'IN_SURDEZ',
           'IN_DISLEXIA',
           'IN_DISCALCULIA',
           'IN_SABATISTA',
           'IN_GESTANTE',
           'IN_IDOSO',
           'TP_PRESENCA_CN',
           'TP_PRESENCA_CH',
           'TP_PRESENCA_LC',
           'CO_PROVA_CN', 
           'CO_PROVA_CH',
           'CO_PROVA_LC',
           'CO_PROVA_MT',
           'NU_NOTA_CN', 
            'NU_NOTA_CH', 
            'NU_NOTA_LC', 
            'TP_LINGUA',
           'TP_STATUS_REDACAO',
            'NU_NOTA_COMP1', 
            'NU_NOTA_COMP2', 
            'NU_NOTA_COMP3',
           'NU_NOTA_COMP4', 
            'NU_NOTA_COMP5', 
            'NU_NOTA_REDACAO']

target = 'NU_NOTA_MT'

cd_cat = ['TP', 'CO', 'SG']
cat_features = [i for i in features if i[:2] in cd_cat] #Variáveis Categóricas
num_features = list( set(features) - set(cat_features)) #Variáveis Numéricas


In [307]:
train = train.dropna(how = 'all', subset=[target])

train[cat_features] = train[cat_features].fillna(-1).astype(str)
train[num_features] = train[num_features].fillna(-1)

train = train.reset_index(drop=True)

In [308]:
onehot = preprocessing.OneHotEncoder(sparse=False, handle_unknown='ignore')
onehot.fit( train[cat_features])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=False)

In [309]:
train_onehot = pd.DataFrame(onehot.transform( train[cat_features]), columns = onehot.get_feature_names(cat_features))

In [310]:
df_train = pd.concat([train[num_features], train_onehot], axis=1, ignore_index=True)
df_train.shape

(10133, 141)

In [311]:
#reg = tree.DecisionTreeRegressor(max_depth = 12, min_samples_leaf=5)
reg = linear_model.LinearRegression()
reg.fit(df_train, train[target])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [312]:
reg.score(df_train, train[target])

0.43962435380137926

In [313]:
features = df_train.columns.tolist()
model = pd.Series([num_features, cat_features, features, reg, onehot], 
                  index = ['num_features', 'cat_features','features', 'reg', 'onehot'])

In [314]:
model.to_pickle('model.pkl')

In [315]:
test = pd.read_csv('test.csv')

In [316]:
modelo = pd.read_pickle('model.pkl')

In [317]:
test[modelo['cat_features']] = test[modelo['cat_features']].fillna(-1).astype(str)
test[modelo['num_features']] = test[modelo['num_features']].fillna(-1)

In [318]:
df_onehot = pd.DataFrame(modelo['onehot'].transform(test[modelo['cat_features']]), 
                        columns = modelo['onehot'].get_feature_names(modelo['cat_features']))

In [319]:
df_full = pd.concat([test[modelo['num_features']], df_onehot],axis=1 ,ignore_index=True)

In [320]:
df_full.isna().sum().sum()

0

In [321]:
predict = modelo['reg'].predict(df_full[model['features']])

In [322]:
df_new = test[['NU_INSCRICAO']]
df_new['NU_NOTA_MT'] = predict

c:\users\miyaz\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [323]:
df_new.to_csv('answer.csv', index = False)